# Вступ до BigQuery ML (BQML)

[BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction) дозволяє використовувати `SQL` для побудови робочого процесу ML.  Це великий стрибок у продуктивності та гнучкості, коли джерелом даних є [BigQuery](https://cloud.google.com/bigquery/docs/introduction) і користувачі вже знайомі з `SQL. Використовуючи лише `SQL`, для навчання моделі можна застосовувати [численні методи](https://cloud.google.com/bigquery/docs/bqml-introduction#model_selection_guide) і навіть включати [налаштування гіперпараметрів](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-hp-tuning-overview).  Він включає безсерверні методи [навчання, оцінювання та виведення](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-e2e-journey) для контрольованих і неконтрольованих методів, методів часових рядів і навіть механізмів рекомендацій.  [Передбачення](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-inference-overview) можуть обслуговуватися безпосередньо у BigQuery, які також включають міри пояснюваності. Предиктивні моделі можна [експортувати у власний фреймворк](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-export-model) для перенесення, або навіть безпосередньо [зареєструвати в реєстрі моделей Vertex AI](https://cloud.google.com/bigquery/docs/managing-models-vertex) для онлайн-прогнозування на кінцевих точках Vertex AI.  Ви можете [імпортувати моделі в BigQuery ML](https://cloud.google.com/bigquery/docs/reference/standard-sql/inference-overview#inference_using_imported_models) з багатьох поширених фреймворків або [підключитися до віддалено розміщених моделей](https://cloud.google.com/bigquery/docs/reference/standard-sql/inference-overview#inference_using_remote_models) на Vertex AI Endpoints. Ви навіть можете безпосередньо використовувати багато [попередньо навчених моделей](https://cloud.google.com/bigquery/docs/reference/standard-sql/inference-overview#pretrained-models) у Vertex AI, таких як Cloud Vision API, Cloud Natural Language API та Cloud Translate API.

**Ви вже знаєте:**
```SQL
CREATE TABLE {table name}
    AS
        SELECT ...
        FROM ...
        WHERE ...
```

**Тепер спробуйте:**
```SQL
CREATE MODEL {model name} 
    OPTIONS (
        MODEL_TYPE = ...
        {more options}
    )
    AS
        SELECT ...
        FROM ...
        WHERE ...
```

**Або навіть так:**
- ```SQL
CREATE OR REPLACE MODEL {model name}...
```
- ```SQL
CREATE MODEL IF NOT EXISTS {model name}...
```

**Функції для вашого робочого процесу ML**
- Оцінка моделі за допомогою `ML.EVALUTE`:
    - ```SQL
    SELECT *
    FROM ML.EVALUTE(
        MODEL {model name},
        (
            SELECT ...
            FROM ...
            WHERE ...),
        STRUCT({optional parameters})
    )
    ```
- Прогнозування за допомогою моделей з `ML.PREDICT`:
    - ```SQL
    SELECT *
    FROM ML.PREDICT(
        MODEL {model name},
        (
            SELECT ...
            FROM ...
            WHERE ...),
        STRUCT({optional parameters})
    )
    ```
---

**Початок знайомства**

Доступні типи моделей і функції, які допоможуть створити робочий процес ML з кожним типом моделі:

- [End-to-end user journey for each model](https://cloud.google.com/bigquery/docs/e2e-journey)
- [Introduction to BigQuery ML](https://cloud.google.com/bigquery/docs/bqml-introduction)

---
## Налаштування

вхідні дані:

In [ ]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

In [ ]:
# початкові дані
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

пакети:

In [ ]:
import pandas as pd
from google.cloud import bigquery

клієнт:

In [ ]:
bq = bigquery.Client()

параметри:

In [ ]:
BQ_SOURCE = f'{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}'

---
## Використання BigQuery з Jupyter та Python

### BigQuery Cell Magic

In [ ]:
%load_ext google.cloud.bigquery

In [ ]:
%%bigquery
SELECT COUNT(*) as n
FROM `fit-cloud-course.fraud.fraud_prepped` # це не можна параметризувати за допомогою магії

### Клієнт BigQuery на Python

In [ ]:
query = f"""
    SELECT splits, COUNT(*) as n 
    FROM `{BQ_SOURCE}`
    GROUP BY splits
"""
preview = bq.query(query = query).to_dataframe()
preview

### Клієнт BigQuery на Python: Використання властивостей та методів запиту

Запити мають методи та властивості, які можуть бути корисними для робочого процесу Python:
- [Методи](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.job.QueryJob#methods)
- [Properties](https://cloud.google.com/python/docs/reference/bigquery/latest/google.cloud.bigquery.job.QueryJob#properties)

BigQuery запит (з використанням допоміжної функції):

In [ ]:
job = bq_runner(
    query = f"""
    WITH
        COUNTS AS (
            SELECT splits, Class, COUNT(*) as n 
            FROM `{BQ_SOURCE}`
            GROUP BY splits, Class
        )
    SELECT *,
        SUM(n) OVER() as total,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY Class)) as n_pct_class,
        SAFE_DIVIDE(n, SUM(n) OVER(PARTITION BY splits)) as n_pct_split,
        SAFE_DIVIDE(SUM(n) OVER(PARTITION BY Class), SUM(n) OVER()) as class_pct_to  
    FROM COUNTS
    """
)

Розрахуємо час:

In [ ]:
job.result()
(job.ended-job.started).total_seconds()

Pandas dataframe:

In [ ]:
job.to_dataframe()

### Непряме використання з pandas-gbq

При роботі з [Pandas](https://pandas.pydata.org/docs/user_guide/index.html#user-guide) наведені вище методи показують, що клієнт повертає дані у вигляді pandas датафрейму.  У цьому розділі буде показано модуль pandas, [pandas-gbq](https://pandas-gbq.readthedocs.io/en/latest/), який обгортає клієнт BigQuery так, щоб pandas міг отримувати дані BigQuery у фрейми даних.

Посилання:
- [Порівняння клієнта BigQuery з pandas-gbq](https://cloud.google.com/bigquery/docs/pandas-gbq-migration)

#### Встановити пакет (якщо потрібно)

In [ ]:
try:
    import pandas_gbq
except ImportError:
    print('Вам потрібно виконати pip install pandas-gbq')
    !pip install pandas-gbq -q

#### Using pandas-gbq

In [ ]:
query = f"""
SELECT * 
FROM `{BQ_SOURCE}`
LIMIT 5
"""
df = pd.read_gbq(query, project_id = BQ_PROJECT)

In [ ]:
df